<a href="https://colab.research.google.com/github/ashleyngomez380/SAMcode/blob/main/protypical_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as nn

In [2]:
data = torch.randn((100,64))

In [3]:
label = torch.cat((torch.zeros((50,1)),torch.ones((50,1)))).int()
label.shape

torch.Size([100, 1])

In [4]:
uniqueL = label.unique()
label_list = uniqueL.squeeze()
label_list

tensor([0, 1], dtype=torch.int32)

In [5]:
# do not use a list for generating even empty array.
# try to use the data strcuture such as torch.zeros, nn.zeros...

In [6]:
def compute_per_class_centroid(i,label,data):
  # i : the target class label
  # label: N x 1 vector stores the label
  # data: N x 64 embedding vector
  # return: 1 x 64 vector which is the sample of average all samples in that class.

  # get rid of the extra dimension for label
  label1d = label.squeeze()

  # given class extracting samples
  data_class = data[label1d==i, :]

  # compute the average of your class sample
  class_centroid  = torch.mean(data_class, 0, True)
  return class_centroid

In [7]:
def compute_class_centroid(label, data):
    # i : the target class label
    # label: N x 1 vector stores the label
    # data: N x 64 embedding vector
    # return: C x 64 vector which is the average of all samples, where C is number of class
    unique_labels = label.unique().squeeze()
    #print(unique_labels)

    # initialize centroids tensor with first centroid
    centroids = compute_per_class_centroid(unique_labels[0], label, data)
    #print("Centroids at ", 0, " = ", centroids.shape)

    # add every other class centroid to centroids tensor
    for i in range(1, ((len(unique_labels)))):
      index = unique_labels[i]
      centroids = torch.cat((centroids, compute_per_class_centroid(index, label, data)), dim=0)
      #print("Centroids at ", i, " = ", centroids.shape)

    return centroids

In [8]:
#compute pairwise distance between centroids and data

# https://pytorch.org/docs/stable/generated/torch.nn.functional.pairwise_distance.html

def distance_matrix(data, centroid):
  # finding number of classes
  #unique_labels = label.unique().squeeze()

  # distance function
  distance = torch.cdist(data,centroid,p=2)

  return distance

In [9]:
distance = distance_matrix(data, result)

NameError: name 'result' is not defined

In [10]:
# Crossentropy over normalized distance
def prototypical_loss(D, labels):
  # D: N x C distance matrix where C is number of the class
  # Output: Loss function value l
  # o = softmax(D)
  # return crossentropy(o)

  # softmax our distance tensor
  softmax = torch.nn.Softmax(dim=1)
  o = softmax(D)
  #print(o)

  # tensor of our classes prediction
  _, predicted_classes = torch.min(D, dim=1)
  #print(predicted_classes)

  # make labels a long data type for CE
  labels = labels.squeeze().long()

  # cross entropy loss
  loss = nn.cross_entropy(D, labels, reduction='mean')

  return loss

In [11]:
loss = prototypical_loss(distance, label)
loss

NameError: name 'distance' is not defined